In [2]:
import os


# Initialize database connection
username = os.getenv("AZURE_SQL_USERNAME")
password = os.getenv("AZURE_SQL_PASSWORD")
server_name = os.getenv("AZURE_SQL_SERVER")
database_name = os.getenv("AZURE_SQL_DATABASE")

connection_string = (
    f"mssql+pyodbc://{username}:{password}@{server_name}"
    f"/{database_name}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&TrustServerCertificate=yes"
    "&timeout=30"
        )

connection_string

'mssql+pyodbc://sa:ModyAtta123@./HumanResources?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes&timeout=30'

In [3]:
from langchain_community.utilities.sql_database import SQLDatabase
database = SQLDatabase.from_uri(connection_string)

In [4]:
tenantUsername = 'mendel-ai'
tenant_query = f"""
SELECT TenantID FROM Tenants WHERE Name = '{tenantUsername}'
"""
result = database.run(tenant_query)
tenant_id = ''.join(filter(str.isdigit, result))

tenant_id

'3258'

In [18]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [20]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]

In [19]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

In [23]:
from langchain import PromptTemplate

table_info = open('databaseSchema.txt').read()
# print(table_info)
new_prompt = PromptTemplate(
    template=   '''
Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}''',
    input_variables=['input', 'table_info']
)


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = new_prompt.invoke(
        {
            "dialect": database.dialect,
            "top_k": 10,
            "table_info": database.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}